# MuJoCo in Multiverse Tutorial

## <span style="color:red">**Important!!! Navigate to the 'Multiverse' directory first**</span>


In [ ]:
if [ "$(basename "$PWD")" == "Multiverse" ]; then
    echo "You are in the 'Multiverse' directory."
else
    echo "You are not in the 'Multiverse' directory, navigate to the 'Multiverse' directory."
    cd ..
    pwd
    if [ "$(basename "$PWD")" == "Multiverse" ]; then
        echo "You are now in the 'Multiverse' directory."
    else
        echo "You are still not in the 'Multiverse' directory..."
    fi
fi
MULTIVERSE_DIR=$PWD

## 0. Clone these repositories and build them

In [ ]:
source $MULTIVERSE_DIR/multiverse_ws/devel/setup.bash
git clone https://github.com/Multiverse-Framework/Multiverse-Robots.git $MULTIVERSE_DIR/multiverse_ws/src/multiverse_robots
git clone https://github.com/Multiverse-Framework/Multiverse-World.git $MULTIVERSE_DIR/multiverse_ws/src/multiverse_world
git clone https://github.com/Multiverse-Framework/Multiverse-Objects.git $MULTIVERSE_DIR/multiverse_ws/src/multiverse_objects
git clone https://github.com/Multiverse-Framework/Multiverse-Demos.git $MULTIVERSE_DIR/multiverse_ws/src/multiverse_demos
cd $MULTIVERSE_DIR/multiverse_ws
catkin build
cd ..

## A. Import a robot from URDF into MuJoCo

### 1. Create a tutorial workspace

In [ ]:
rm -rf $MULTIVERSE_DIR/tutorial_ws
mkdir -p $MULTIVERSE_DIR/tutorial_ws/src

### 2. Create `my_robot_description` package that stores all descriptions of the robot

In [ ]:
source $MULTIVERSE_DIR/multiverse_ws/devel/setup.bash

cd $MULTIVERSE_DIR/tutorial_ws/src
catkin_create_pkg my_robot_description
cd ..
catkin build
cd ..

### 3. Use `multiverse_parser` to clean up and convert the robot from URDF to MJCF

In [ ]:
multiverse_parser --help

#### a) Clean up the URDF

##### Convert all *.dae to *.obj and repath the meshes

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

TIAGO_DUAL_URDF=$(rospack find tiago_dual_description)/urdf/tiago_dual.urdf
MY_ROBOT_URDF=$(rospack find my_robot_description)/urdf/my_robot.urdf

multiverse_parser --input=$TIAGO_DUAL_URDF --output=$MY_ROBOT_URDF

##### Test the URDF

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

roslaunch multiverse_parser view_urdf.launch urdf:=$MY_ROBOT_URDF

#### b) Convert the robot from URDF into MJCF

##### Use `--no-visual` to drop all visual meshes from URDF

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

TIAGO_DUAL_URDF=$(rospack find tiago_dual_description)/urdf/tiago_dual.urdf
MY_ROBOT_MJCF=$(rospack find my_robot_description)/mjcf/my_robot.xml

multiverse_parser --input=$TIAGO_DUAL_URDF --output=$MY_ROBOT_MJCF --no-visual

##### Test the MJCF

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ROBOT_MJCF=$(rospack find my_robot_description)/mjcf/my_robot.xml

simulate $MY_ROBOT_MJCF

##### Insert this tag into `my_robot.xml`, before `</mujoco>` to exclude the contact pairs

```xml
<contact>
    <exclude body1="torso_lift_link" body2="torso_fixed_column_link"/>
    <exclude body1="base_link" body2="wheel_right_link"/>
    <exclude body1="base_link" body2="wheel_left_link"/>
    <exclude body1="base_link" body2="caster_back_left_2_link"/>
    <exclude body1="base_link" body2="caster_back_right_2_link"/>
    <exclude body1="base_link" body2="caster_front_left_2_link"/>
    <exclude body1="base_link" body2="caster_front_right_2_link"/>
</contact>
```

#### c) Clean up URDF and MJCF if needed
- Change robot name
- Add some parameters

## B. Control the robot in MuJoCo with ROS

### 1. Create `my_robot_mujoco` package to launch the robot from ROS

#### a) Create the package and build it

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

cd $MULTIVERSE_DIR/tutorial_ws/src
catkin_create_pkg my_robot_mujoco
mkdir my_robot_mujoco/launch
mkdir my_robot_mujoco/config
touch my_robot_mujoco/config/mujoco_params.yaml
touch my_robot_mujoco/config/control_params.yaml
touch my_robot_mujoco/config/multiverse_params.yaml
touch my_robot_mujoco/launch/my_robot_mujoco.launch
cd ..
catkin build
cd ..

#### b) Copy this configuration into `mujoco_params.yaml`

```yaml
mujoco:
  robots: [my_robot]
  pose_init: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
  add_odom_joints: true
  root_frame_id: map
```

#### c) Copy this launch file into `my_robot_mujoco.launch`

```xml
<launch>

    <include file="$(find mujoco_sim)/launch/mujoco_sim.launch">
        <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
        <arg name="robot" value="$(find my_robot_description)/mjcf/my_robot.xml" />
    </include>

</launch>
```

#### d) Launch the robot in MuJoCo from ROS

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

roslaunch my_robot_mujoco my_robot_mujoco.launch

### 2. Add controller for the robot

#### a) Copy the following configuration into `control_params.yaml`

```yaml
joint_state_controller:
  type: joint_state_controller/JointStateController
  publish_rate: 60

torso_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - torso_lift_joint
  gains:
    torso_lift_joint: &pid_gains {p: 2000, i: 100, d: 50}

head_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - head_1_joint
    - head_2_joint
  gains:
    head_1_joint: *pid_gains
    head_2_joint: *pid_gains

arm_left_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - arm_left_1_joint
    - arm_left_2_joint
    - arm_left_3_joint
    - arm_left_4_joint
    - arm_left_5_joint
    - arm_left_6_joint
    - arm_left_7_joint
  gains:
    arm_left_1_joint: *pid_gains
    arm_left_2_joint: *pid_gains
    arm_left_3_joint: *pid_gains
    arm_left_4_joint: *pid_gains
    arm_left_5_joint: *pid_gains
    arm_left_6_joint: *pid_gains
    arm_left_7_joint: *pid_gains

arm_right_controller:
  type: "effort_controllers/JointTrajectoryController"
  joints:
    - arm_right_1_joint
    - arm_right_2_joint
    - arm_right_3_joint
    - arm_right_4_joint
    - arm_right_5_joint
    - arm_right_6_joint
    - arm_right_7_joint
  gains:
    arm_right_1_joint: *pid_gains
    arm_right_2_joint: *pid_gains
    arm_right_3_joint: *pid_gains
    arm_right_4_joint: *pid_gains
    arm_right_5_joint: *pid_gains
    arm_right_6_joint: *pid_gains
    arm_right_7_joint: *pid_gains

gripper_left_left_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_left_left_finger_joint
gripper_left_right_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_left_right_finger_joint
gripper_right_left_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_right_left_finger_joint
gripper_right_right_finger_controller:
  type: effort_controllers/JointEffortController
  joint: gripper_right_right_finger_joint
```

#### b) Insert the following code into `my_robot_mujoco.launch`

```xml
<param name="robot_description" command="$(find xacro)/xacro '$(find my_robot_description)/urdf/my_robot.urdf'" />
    <rosparam command="load" file="$(find my_robot_mujoco)/config/control_params.yaml" />
    <node name="controller_manager" pkg="controller_manager" type="controller_manager" respawn="false" output="screen" args="spawn 
        joint_state_controller
        torso_controller
        head_controller
        arm_left_controller
        arm_right_controller
        gripper_left_left_finger_controller
        gripper_left_right_finger_controller
        gripper_right_right_finger_controller
        gripper_right_left_finger_controller" />
    <node name="rqt_joint_trajectory_controller" pkg="rqt_joint_trajectory_controller" type="rqt_joint_trajectory_controller" />
    <node name="rqt_robot_steering" pkg="rqt_robot_steering" type="rqt_robot_steering" />
```

#### c) Launch the robot with controllers

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

roslaunch my_robot_mujoco my_robot_mujoco.launch

## C. Connect MuJoCo to ROS through Multiverse Server

### 1. Connect MuJoCo to Multiverse Server

#### Insert this configuration into `mujoco_params.yaml`

```yaml
multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs
  
  send:
    body: [position, quaternion]
    joint: [joint_rvalue, joint_tvalue]
    
  receive:
    my_robot: [relative_velocity]
```

### 2. Create Multiverse Client for ROS

#### a) Copy the following configuration into `multiverse_params.yaml`

```yaml
multiverse:  
  length_unit: m
  angle_unit: rad
  force_unit: N
  time_unit: s
  handedness: rhs

  publishers:
    tf:
      port: 7300
      rate: 60

  subscribers:
    cmd_vel:
      port: 7200
      body: my_robot
```

#### b) Insert the following code into `my_robot_mujoco.launch`

```xml
<rosparam command="load" file="$(find my_robot_mujoco)/config/multiverse_params.yaml" />
<node pkg="multiverse_socket" type="multiverse_socket_node.py" name="multiverse_socket" output="screen" />
<node pkg="rviz" type="rviz" name="rviz" />
```

### 3. Run `multiverse_server` in another terminal

In [ ]:
gnome-terminal -- multiverse_server

### 4. Launch the robot in Multiverse (keep `multiverse_server` running)

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

roslaunch my_robot_mujoco my_robot_mujoco.launch

## D. Import an environment from USD into MuJoCo

### 1. Create `my_env_description` package that stores all descriptions of the environment

In [ ]:
source $MULTIVERSE_DIR/multiverse_ws/devel/setup.bash

cd $MULTIVERSE_DIR/tutorial_ws/src
catkin_create_pkg my_env_description
cd ..
catkin build
cd ..

### 2. Use `multiverse_parser` to convert the environment from USD into URDF and into MJCF

#### a) Check the environment in USD

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ENV_IN_USD=$(rospack find iai_apartment)/usd/apartment.usda
usdview $MY_ENV_IN_USD

#### b) Convert the environment from USD into URDF and into MJCF

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ENV_IN_USD=$(rospack find iai_apartment)/usd/apartment.usda
MY_ENV_IN_URDF=$(rospack find my_env_description)/urdf/my_environment.urdf
MY_ENV_IN_MJCF=$(rospack find my_env_description)/mjcf/my_environment.xml

#multiverse_parser --input=$MY_ENV_IN_USD --output=$MY_ENV_IN_URDF
multiverse_parser --input=$MY_ENV_IN_USD --output=$MY_ENV_IN_MJCF --no-visual

#### c) Add floor and light to the environment in MJCF

```xml
<asset>
    <texture name="grid" type="2d" builtin="checker" width="512" height="512" rgb1=".1 .2 .3" rgb2=".2 .3 .4" />
    <material name="grid" texture="grid" texrepeat="1 1" texuniform="true" reflectance=".2" />
</asset>

<worldbody>
    <geom name="floor" size="0 0 .05" type="plane" material="grid" condim="4" friction="2 0.05 0.01" />
    <light diffuse=".5 .5 .5" pos="0 0 5" dir="0 0 -1" />
</worldbody>
```

#### d) Check the environment in URDF and MJCF

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

MY_ENV_IN_URDF=$(rospack find my_env_description)/urdf/my_environment.urdf
MY_ENV_IN_MJCF=$(rospack find my_env_description)/mjcf/my_environment.xml

roslaunch multiverse_parser view_urdf.launch urdf:=$MY_ENV_IN_URDF & simulate $MY_ENV_IN_MJCF

#### d) Clean up the URDF and MJCF if needed
- Change environment name
- Add some parameters

### 3. Use the environment in MuJoCo

#### a) Replace the following code in `my_robot_mujoco.launch`

```xml
<param name="env_description" command="$(find xacro)/xacro '$(find my_env_description)/urdf/my_environment.urdf'" />
<include file="$(find mujoco_sim)/launch/mujoco_sim.launch">
    <arg name="config" value="$(find my_robot_mujoco)/config/mujoco_params.yaml" />
    <arg name="robot" value="$(find my_robot_description)/mjcf/my_robot.xml" />
    <arg name="world" value="$(find my_env_description)/mjcf/my_environment.xml" />
</include>
```

#### b) Change the `pose_init` in `mujoco_params.yaml`

```yaml
pose_init: [1.5, 2.5, 0.0, 0.0, 0.0, 0.0]
```

#### c) Launch the robot with environment in Multiverse (keep `multiverse_server` running)

In [ ]:
source $MULTIVERSE_DIR/tutorial_ws/devel/setup.bash

roslaunch my_robot_mujoco my_robot_mujoco.launch